In [1]:
# assemble data from yahoo finance api
# install these dependencies
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# Define the stock symbol and date range
ticker_symbol = 'TSLA'

# Download daily data for Tesla from 2015 to 2023
tesla_daily = yf.download(ticker_symbol, start="2015-01-01", end="2023-10-01", interval="1d")

# Display the first few rows
print(tesla_daily.head())

[*********************100%***********************]  1 of 1 completed

Price                      Adj Close      Close       High        Low  \
Ticker                          TSLA       TSLA       TSLA       TSLA   
Date                                                                    
2015-01-02 00:00:00+00:00  14.620667  14.620667  14.883333  14.217333   
2015-01-05 00:00:00+00:00  14.006000  14.006000  14.433333  13.810667   
2015-01-06 00:00:00+00:00  14.085333  14.085333  14.280000  13.614000   
2015-01-07 00:00:00+00:00  14.063333  14.063333  14.318667  13.985333   
2015-01-08 00:00:00+00:00  14.041333  14.041333  14.253333  14.000667   

Price                           Open    Volume  
Ticker                          TSLA      TSLA  
Date                                            
2015-01-02 00:00:00+00:00  14.858000  71466000  
2015-01-05 00:00:00+00:00  14.303333  80527500  
2015-01-06 00:00:00+00:00  14.004000  93928500  
2015-01-07 00:00:00+00:00  14.223333  44526000  
2015-01-08 00:00:00+00:00  14.187333  51637500  


In [3]:
# Generate a complete daily date range for the period covered by tesla_daily
expected_daily_range = pd.date_range(start=tesla_daily.index.min(), end=tesla_daily.index.max(), freq='D')

# Identify any missing dates by comparing with the daily data's index
missing_days = expected_daily_range.difference(tesla_daily.index)

# Calculate missing data statistics
missing_count = len(missing_days)
total_days = len(expected_daily_range)
missing_percentage = (missing_count / total_days) * 100

missing_count, total_days, missing_percentage



(992, 3193, 31.06796116504854)

In [4]:
tesla_daily_interpolated = tesla_daily.interpolate(method='linear')


In [5]:
# Generate a complete date range for the daily frequency
expected_daily_range = pd.date_range(start=tesla_daily.index.min(), end=tesla_daily.index.max(), freq='D')

# Check for missing dates after filling
missing_days_after_fill = expected_daily_range.difference(tesla_daily_interpolated.index)
is_data_complete = len(missing_days_after_fill) == 0

# Display column names
columns = tesla_daily_interpolated.columns

is_data_complete, columns


(False,
 MultiIndex([('Adj Close', 'TSLA'),
             (    'Close', 'TSLA'),
             (     'High', 'TSLA'),
             (      'Low', 'TSLA'),
             (     'Open', 'TSLA'),
             (   'Volume', 'TSLA')],
            names=['Price', 'Ticker']))

In [6]:
# Generate expected full daily range for the time frame
expected_daily_range = pd.date_range(start=tesla_daily_interpolated.index.min(), end=tesla_daily_interpolated.index.max(), freq='D')

# Identify missing dates filling
missing_days_after_fill = expected_daily_range.difference(tesla_daily_interpolated.index)

# Display the list of missing dates
missing_days_after_fill


DatetimeIndex(['2015-01-03 00:00:00+00:00', '2015-01-04 00:00:00+00:00',
               '2015-01-10 00:00:00+00:00', '2015-01-11 00:00:00+00:00',
               '2015-01-17 00:00:00+00:00', '2015-01-18 00:00:00+00:00',
               '2015-01-19 00:00:00+00:00', '2015-01-24 00:00:00+00:00',
               '2015-01-25 00:00:00+00:00', '2015-01-31 00:00:00+00:00',
               ...
               '2023-08-27 00:00:00+00:00', '2023-09-02 00:00:00+00:00',
               '2023-09-03 00:00:00+00:00', '2023-09-04 00:00:00+00:00',
               '2023-09-09 00:00:00+00:00', '2023-09-10 00:00:00+00:00',
               '2023-09-16 00:00:00+00:00', '2023-09-17 00:00:00+00:00',
               '2023-09-23 00:00:00+00:00', '2023-09-24 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=992, freq=None)

In [7]:
# Reindex the DataFrame to include all dates range
tesla_daily_filled = tesla_daily.reindex(expected_daily_range)

# Forward fill to propagate non-trading days
tesla_daily_filled = tesla_daily_filled.ffill()

# checking for any remaining missing dates
missing_days_after_fill = expected_daily_range.difference(tesla_daily_filled.index)
is_data_complete = len(missing_days_after_fill) == 0

is_data_complete, tesla_daily_filled.head()


(True,
 Price                      Adj Close      Close       High        Low  \
 Ticker                          TSLA       TSLA       TSLA       TSLA   
 2015-01-02 00:00:00+00:00  14.620667  14.620667  14.883333  14.217333   
 2015-01-03 00:00:00+00:00  14.620667  14.620667  14.883333  14.217333   
 2015-01-04 00:00:00+00:00  14.620667  14.620667  14.883333  14.217333   
 2015-01-05 00:00:00+00:00  14.006000  14.006000  14.433333  13.810667   
 2015-01-06 00:00:00+00:00  14.085333  14.085333  14.280000  13.614000   
 
 Price                           Open      Volume  
 Ticker                          TSLA        TSLA  
 2015-01-02 00:00:00+00:00  14.858000  71466000.0  
 2015-01-03 00:00:00+00:00  14.858000  71466000.0  
 2015-01-04 00:00:00+00:00  14.858000  71466000.0  
 2015-01-05 00:00:00+00:00  14.303333  80527500.0  
 2015-01-06 00:00:00+00:00  14.004000  93928500.0  )

In [8]:
# Feature engineering
# remove long annoying date column but extract useful features
# Extracting day of the week and month as features
# If MultiIndex, (like its labeled Close, TSLA, for the column headers, we just want it to be Close)
tesla_daily_filled.columns = tesla_daily_filled.columns.get_level_values(0)

# Add day of the week, 0-6
tesla_daily_filled['Day_of_Week'] = tesla_daily_filled.index.dayofweek

# Add month, 1-12
tesla_daily_filled['Month'] = tesla_daily_filled.index.month

# Reset the index to remove the date
tesla_daily_filled = tesla_daily_filled.reset_index(drop=True)

In [9]:

# Drop the second row (index 1)
tesla_daily_filled = tesla_daily_filled.drop(index=1)
# Assuming 'tesla_daily_filled' is the forward-filled DataFrame
tesla_daily_filled.to_csv("tesla_daily_filled.csv")

# Add a 30-day moving average to approximate the last month's trend
tesla_daily_filled['MA_30'] = tesla_daily_filled['Close'].rolling(window=30).mean()

# Calculate Target_Close by shifting 'Close' column by -1
tesla_daily_filled['Target_Close'] = tesla_daily_filled['Close'].shift(-1)

# Calculate Price_Movement as the difference in price for the next day
tesla_daily_filled['Price_Movement'] = tesla_daily_filled['Target_Close'] - tesla_daily_filled['Close']

# Drop the last row if it has NaN values in 'Target_Close' (bc of shifting)
tesla_daily_filled = tesla_daily_filled.dropna(subset=['Target_Close'])

# Reset index to create a clean column for the first row and rename it to 'Time_Step'
tesla_daily_filled = tesla_daily_filled.reset_index(drop=True)
tesla_daily_filled.index.name = 'Time_Step'  # This sets the first column name in the DataFrame

# Reset the index to turn 'Time_Step' from an index into a column
tesla_daily_filled = tesla_daily_filled.reset_index()

# convert to csv to be loaded for training
tesla_daily_filled.to_csv("tesla_data.csv", index=False)

Current index: RangeIndex(start=0, stop=3191, step=1)
